In [121]:
#IMPORTACIÓN DE LIBRERÍAS

import numpy as np
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from scipy.stats import poisson

In [115]:
class DataGetter:
    '''
    Clase encargada de obtener los partidos de la Copa America 2024 y el Ranking Mundial de la FIFA actual.
    
    Atributos:
        url: path de la página Web.

    Métodos:
        charger: Se encarga de leer el path y entregar una lista tratable con pandas.
        organicer: Se encargada de organizar, filtrar y dejar listos los datos cargados. 
    '''
    def __init__(self, url):
        self.link = url #Link de la página web a la que accedemos
    
    def charger(self):
        data = pd.read_html(self.link) #Leemos la página web
        
        return data

    def organicer(self, data, mode = 'CA'):
        if mode == 'CA': #Si mode = CA indica que organizamos los datos de las tablas de la Copa América
            dict_tables = {} #Creamos un diccionario vacío para rellenarlo con las tablas
            for letra, i in zip(alfabeto, range(14, 42, 7)): #Según el patrón encontrado, iteramos
                df = data[i] #Definimos el DataFrame para cada elemento encontrado en la web
                df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
                df.pop('Qualification') #Borramos columnas innecesarias
                dict_tables[f'Group {letra}'] = df #Definimos el nombre de cada grupo (A, B, C, D)
            
            for group in dict_tables:
                dict_tables[group]['Pts'] = 0

            #Renombramos algunas selecciones que presentaron problemas
            dict_tables['Group C'].loc[2, 'Team'] = 'United States'
            dict_tables['Group D'].loc[0, 'Team'] = 'Colombia'
            dict_tables['Group D'].loc[3, 'Team'] = 'Paraguay'

            
            return dict_tables
        
        elif mode == 'RM': #Si mode = RM indica que organizamos los datos del ranking mundial de selecciones
            
            #Definimos los DataFrame de los ranking de acuerdo al patrón encontrado en la lectura de la Web
            historical_ranking = data[3]
            actual_ranking = data[0]

            #Leyendo los datos, analizamos columnas innecesarias que deben ser eliminadas
            actual_ranking = actual_ranking.drop(0)
            actual_ranking = actual_ranking.drop(1)
            actual_ranking = actual_ranking.drop(2)
            actual_ranking = actual_ranking.drop(23)
            actual_ranking = actual_ranking.drop(24)

            #Renombramos las columnas para que tengan nombres adecuados
            actual_ranking.rename(columns={actual_ranking.columns[0]: 'Rank'}, inplace=True)
            actual_ranking.rename(columns={actual_ranking.columns[1]: 'Change'}, inplace=True)
            actual_ranking.rename(columns={actual_ranking.columns[2]: 'Team'}, inplace=True)
            actual_ranking.rename(columns={actual_ranking.columns[3]: 'Points'}, inplace=True)

            actual_ranking.pop('Change') #Eliminamos la columna que nos da info irrelevante

            return actual_ranking, historical_ranking

In [116]:
#Links con la info necesaria
url_ca = 'https://en.wikipedia.org/wiki/2024_Copa_America'
url_rm = "https://en.wikipedia.org/wiki/FIFA_Men's_World_Ranking"

#Creamos los objetos correspondientes
Copa_America = DataGetter(url_ca)
Ranking_Mundial = DataGetter(url_rm)

#Cargamos los datos
datos_ca = Copa_America.charger()
datos_rm = Ranking_Mundial.charger()

#Organizamos y presentamos los datos
tabla_ca = Copa_America.organicer(datos_ca)
tabla_rm = Ranking_Mundial.organicer(datos_rm, mode = 'RM')

In [117]:
with open('tabla_ca', 'wb') as output:
    pickle.dump(tabla_ca, output)

In [118]:
class DataCollector:

    '''
    Clase diseñada para recolectar los datos de los partidos de cada edición de la Copa America 
    (datos históricos)

    Métodos: 
        
        - get_matches: Se encarga de utilizar WebScraping con bs4 para obtener los datos de los partidos
        de la Copa América en una edición específica.

        - getTotalMatches: Se encarga de recolectar todos los datos históricos de los partidos de la
        Copa América (todas las ediciones).

        -getMissingMatches: Se encarga de utilizar WebScraping con Selenium para obtener los datos faltantes,
        que no pudieron ser recolectados con bs4.

        -getTotalMissingMatches: Se encarga de recolectar los datos faltantes de todas las ediciones que no 
        pudieron ser obtenidas con bs4 (en este caso, eidiciones 2011 y 2015).

        
    '''
 
    def get_matches(self, year):
        
        if year <= 1967:
            urls = f'https://en.wikipedia.org/wiki/{year}_South_American_Championship'
            
        else:
            urls = f'https://en.wikipedia.org/wiki/{year}_Copa_America'
            
        response = requests.get(urls)
        content = response.text
        
        soup = BeautifulSoup(content, 'lxml')
        matches = soup.find_all('div', class_="footballbox")

        home = []
        score = []
        away = []
        
        for match in matches:
            home.append(match.find('th', class_="fhome").get_text())
            score.append(match.find('th', class_="fscore").get_text())
            away.append(match.find('th', class_="faway").get_text())
            
        dict_America = {'home':home, 
                        'score':score,
                        'away':away}

        df_America = pd.DataFrame(dict_America)
        df_America['year'] = year

        return df_America
    
    def getTotalMatches(self, years):

        TotalMatches = [self.get_matches(year) for year in years]
        df_TotalMatches = pd.concat(TotalMatches, ignore_index=True)

        df_conmebol = df_TotalMatches[df_TotalMatches['year'] != 2024]
        df_fixture = df_TotalMatches[df_TotalMatches['year'] == 2024]

        return df_conmebol, df_fixture
    
    def getMissingMatches(self, year):
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        
        url = f'https://en.wikipedia.org/wiki/{year}_Copa_America'
        
        driver.get(url) #Habilitamos el acceso al driver 
        
        missing_matches = driver.find_elements(by='xpath', value='//tr[@style="font-size:90%"]')

        home = []
        score = []
        away = []

        for match in missing_matches:
            home.append(match.find_element(by='xpath', value='./td[1]').text)
            score.append(match.find_element(by='xpath', value='./td[2]').text)
            away.append(match.find_element(by='xpath', value='./td[3]').text)

        dict_missing = {'home':home, 
                    'score':score,
                    'away':away}

        df_missing = pd.DataFrame(dict_missing)
        df_missing['year'] = year
        time.sleep(1) #Tiempo de espera para pasar de una página a la otra

        return df_missing
    
    def getTotalMissingMatches(self, years):
        missing_data = [self.getMissingMatches(year) for year in years]

        # driver.quit() #Línea necesaria para que el driver deje de controlar la página web

        df_missing_data = pd.concat(missing_data, ignore_index=True)

        return df_missing_data

In [119]:
years_America = [1916, 1917, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1929, 1935, 1937, 1939, 1941,
                 1942, 1945, 1946, 1947, 1949, 1953, 1955, 1956, 1957, 1963, 1967, 1975, 1979, 1983, 1987, 1989,
                 1991, 1993, 1995, 1997, 1999, 2001, 2004, 2007, 2011, 2015, 2016, 2019, 2021, 2024]

missing_years = [2011, 2015]

Partidos = DataCollector()
tablas = Partidos.getTotalMatches(years_America)
tablas_perdidas = Partidos.getTotalMissingMatches(missing_years)

In [120]:
tablas[0].to_csv('Conmebol_Copa_America_initial_data.csv', index=False)
tablas[1].to_csv('Programacion_Copa_America_2024.csv', index=False)
tablas_perdidas.to_csv('Conmebol_Copa_America_missing_data.csv', index=False)

In [109]:
class DataCleaner:
    '''
    Clase encargada de limpiar los datos obtenidos usando WebScraping.
    
    Atributos:
        - archivei con i = 1, 2, 3: Son los path de los archivos .csv con los datos necesarios
        
    Métodos:
        - openData: Se encarga de convertir los archivos .csv en un DataFrame de pandas.

        - cleanData: Se encarga de limpiar y transformar los datos para el uso adecuado.
    '''
    def __init__(self, archive1, archive2, archive3):
        self.path1 = archive1
        self.path2 = archive2
        self.path3 = archive3
    
    def openData(self):
        data1 = pd.read_csv(self.path1)
        data2 = pd.read_csv(self.path2)
        data3 = pd.read_csv(self.path3)
        
        return data1, data2, data3
    
    def cleanData(self, data, mode = 0):
        if mode == 0: #Clean Fixture
            data[0]['home'] = data[0].home.str.strip()
            data[0]['away'] = data[0].away.str.strip()

            for i in range(32):
                data[0].loc[i, 'score'] = f'Match {i+1}'

                data[0].loc[24, 'home'] = 'Winner Group A'
                data[0].loc[24, 'away'] = 'Runner-up Group B'
                data[0].loc[25, 'home'] = 'Winner Group B'
                data[0].loc[25, 'away'] = 'Runner-up Group A'
                data[0].loc[26, 'home'] = 'Winner Group D'
                data[0].loc[26, 'away'] = 'Runner-up Group C'
                data[0].loc[27, 'home'] = 'Winner Group C'
                data[0].loc[27, 'away'] = 'Runner-up Group D'
                
                data[0].loc[28, 'home'] = 'Winner Match 25'
                data[0].loc[28, 'away'] = 'Winner Match 26'
                data[0].loc[29, 'home'] = 'Winner Match 27'
                data[0].loc[29, 'away'] = 'Winner Match 28'

                data[0].loc[30, 'home'] = 'Loser Match 29'
                data[0].loc[30, 'away'] = 'Loser Match 30'

                data[0].loc[31, 'home'] = 'Winner Match 29'
                data[0].loc[31, 'away'] = 'Winner Match 30'



            return data[0]
        
        elif mode == 1:
            df_complete_data = pd.concat([data[1], data[2]], ignore_index=True)
            df_complete_data.drop_duplicates(inplace=True)
            df_complete_data.sort_values('year', inplace=True)

            df_complete_data['score'] = df_complete_data['score'].str.strip()
            df_complete_data['score'] = df_complete_data['score'].str.replace('[^\d–]', '', regex=True)
            
            df_complete_data['home'] = df_complete_data.home.str.strip()
            df_complete_data['away'] = df_complete_data.away.str.strip()

            df_complete_data[['home_goals', 'away_goals']] = df_complete_data['score'].str.split('–', expand=True)
            df_complete_data.drop('score', axis=1, inplace=True)

            df_complete_data = df_complete_data.astype({'home_goals': int, 'away_goals':int, 'year':int})

            df_home =  df_complete_data[['home', 'home_goals', 'away_goals']]
            df_away =  df_complete_data[['away', 'home_goals', 'away_goals']]

            #Renombrar Columnas
            df_home = df_home.rename(columns={'home':'Team', 'home_goals':'Goals_Scored', 'away_goals':'Goals_Conceded'})
            df_away = df_away.rename(columns={'away':'Team', 'home_goals':'Goals_Conceded', 'away_goals':'Goals_Scored'}) 

            return df_complete_data, df_home, df_away
    
    def teamStrength(self, df_h, df_a):
        df_team_strength = pd.concat([df_h, df_a], ignore_index=True).groupby('Team').mean()
        return df_team_strength

In [110]:
Tablas = DataCleaner('Programacion_Copa_America_2024.csv',
                    'Conmebol_Copa_America_initial_data.csv',
                    'Conmebol_Copa_America_missing_data.csv')

data = Tablas.openData()
df_fixture = Tablas.cleanData(data)
df_data = Tablas.cleanData(data, 1)
df_complete_data = df_data[0]
df_home = df_data[1]
df_away = df_data[2]
df_strength = Tablas.teamStrength(df_home, df_away)

In [124]:
class Modeling:
    def __init__(self):
        pass

    def Rounds(self, df_f):
        df_fixture_groups = df_f[:24].copy()
        df_fixture_quarter = df_f[24:28].copy()
        df_fixture_semi = df_f[28:30].copy()
        df_fixture_final = df_f[30:].copy()

        return df_fixture_groups, df_fixture_quarter, df_fixture_semi, df_fixture_final

    def predict_points(self, df_s, home, away):
        if home in df_s.index and away in df_s.index:
            #goals_scored*goals_conceded
            lamb_home = df_s.at[home, 'Goals_Scored'] * df_s.at[away, 'Goals_Conceded']
            lamb_away = df_s.at[away, 'Goals_Scored'] * df_s.at[home, 'Goals_Conceded']
            prob_home, prob_away, prob_draw = 0, 0, 0

            for i in range(11): #Goles equipo local
                for j in range(11): #Goles equipo visitante
                    p = poisson.pmf(i, lamb_home) * poisson.pmf(j, lamb_away)
                    if i == j:
                        prob_draw += p
                    elif i > j:
                        prob_home += p
                    else:
                        prob_away += p
        
            points_home = 3 * prob_home + prob_draw
            points_away = 3 * prob_away + prob_draw

            return (points_home, points_away)
        else:
            return (0,0)

In [97]:
df_fixture.to_csv('Programacion_Copa_America_2024.csv', index=False)
df_complete_data.to_csv('Conmebol_Copa_America_complete_data.csv', index=False)

In [99]:
df_data = pd.read_csv('Conmebol_Copa_America_complete_data.csv')
df_fixture = pd.read_csv('Programacion_Copa_America_2024.csv')